#### **Introduction**

This project generates a synthetic dataset of 250,000 adults, designed to reflect the realistic health metrics and distributions of the adult U.S. population, closely following data trends from NHANES (National Health and Nutrition Examination Survey) as provided by the CDC. Initially formulated from a 10,000-row synthetic dataset, this larger dataset was created to support machine learning and advanced data engineering projects that require real-world health data distributions. By employing a series of novel algorithms, the dataset includes essential metabolic health variables such as Age, BMI, Waist Circumference, Fasting Blood Glucose (FBG), Triglycerides, HDL Cholesterol, and High Blood Pressure. These variables were generated with careful attention to accuracy and realism, making the dataset a valuable tool for research, training, and testing in health-related fields.

Dataset Generation Process
Age and BMI Distribution: The foundation of the dataset mimics the age and BMI distribution of the U.S. adult population, segmented into age groups reflecting real-world demographic proportions. BMI values were calculated for each individual based on these groups and modified by gender-specific adjustments, creating a realistic representation of body mass in relation to age and gender.

Waist Circumference Generation: A function was developed to generate waist circumference (WC) values using BMI, age, and gender. Linear relationships and age adjustment factors were applied to ensure that the WC values accurately reflected individual variability in body composition.

Fasting Blood Glucose (FBG) Simulation: The FBG values were calculated using a probabilistic approach based on the individual's BMI and age, aligning the results with realistic thresholds for normal and elevated FBG levels.

Triglycerides and HDL Levels: Similarly, triglyceride levels and HDL cholesterol were calculated using stratified algorithms that adjusted values based on BMI category, age, and gender. These lipid variables are crucial in assessing metabolic syndrome risk and were designed to match realistic health profiles.

High Blood Pressure Assignment: High blood pressure (hypertension) status was assigned based on BMI and age, again using probabilistic models to reflect realistic health trends in the U.S. adult population.

Practical Applications of the Dataset
This project was formulated with the goal of creating a robust dataset that can be used for machine learning and advanced data engineering tasks. By incorporating real-world data distributions, the dataset holds significant value for various applications:

Realistic and Verifiable Distributions: Unlike many synthetic datasets that are purely random, this dataset is built on relationships and distributions that mirror actual healthcare data. This makes it ideal for training and testing machine learning models, as well as for performing simulations intended to reflect real-world health outcomes.

Health Research and Data Science Use Cases: The scarcity of accessible health-related datasets due to privacy concerns (such as HIPAA in the U.S.) makes this synthetic dataset a vital alternative for researchers and data engineers. Its realism ensures that it can be used to develop predictive models and test algorithms without relying on private, sensitive health information.

Ideal for Machine Learning: The large size and diversity of the dataset make it an excellent training ground for machine learning models. It allows organizations and researchers to simulate and validate their healthcare algorithms, helping them improve the accuracy and generalizability of their models.

Addressing a Gap in Availability: There is a significant gap in the availability of large, health-related synthetic datasets that not only preserve privacy but also maintain real-world data distributions. This project fills that gap, making it a valuable resource for organizations involved in healthcare, public health, insurance, and academic research.

In summary, this dataset was designed to provide a scalable, realistic alternative to real-world health data, offering researchers, machine learning practitioners, and data engineers the ability to analyze, predict, and model metabolic health conditions without the limitations of privacy constraints.

#### **Efficient Batch Generation of a 250,000-Row Dataset**

The batch-processing strategy was successfully employed to generate 250,000 rows of synthetic data in 50 batches, with each batch consisting of 5,000 rows. The entire process took 558.28 seconds (approximately 9 minutes and 18 seconds). This method was significantly more efficient than previous attempts, where attempting to generate the entire dataset at once resulted in excessive run times and memory issues due to system constraints.

By dividing the task into smaller, manageable batches, the system resources were better utilized, avoiding memory bottlenecks. This batch-processing approach allowed the code to execute smoothly, ensuring that each batch could be processed and saved independently, without overwhelming the 8 GB memory capacity of the system.

Previous attempts where the entire dataset was generated in one go can be referred to as single-pass generation. These attempts, while conceptually straightforward, were less efficient due to the large memory demands and processing time required. The batch-processing approach proved far more scalable and practical for generating large datasets on systems with moderate resources.

This method allows for flexibility, enabling the dataset size to scale while maintaining performance.








In [ ]:
import numpy as np
import pandas as pd
import time

# Seed for reproducibility
np.random.seed(42)

# Define age groups and their approximate proportions in the adult population
age_groups = {
    (18, 24): 0.12,
    (25, 34): 0.18,
    (35, 44): 0.16,
    (45, 54): 0.15,
    (55, 64): 0.16,
    (65, 74): 0.13,
    (75, 100): 0.10
}

def generate_bmi_vectorized(ages, genders):
    bmi_characteristics = {
        (18, 24): {"mean": 26.3, "std": 4.4},
        (25, 34): {"mean": 27.5, "std": 4.9},
        (35, 44): {"mean": 28.8, "std": 5.3},
        (45, 54): {"mean": 29.3, "std": 5.3},
        (55, 64): {"mean": 29.5, "std": 5.2},
        (65, 74): {"mean": 29.2, "std": 5.0},
        (75, 100): {"mean": 28.3, "std": 4.5}
    }
    
    bmis = np.zeros_like(ages, dtype=float)
    for (lower, upper), group in bmi_characteristics.items():
        mask = (ages >= lower) & (ages <= upper)
        mean = group['mean']
        std = group['std']
        
        gender_adjustment = np.where(genders[mask] == 'Male', 0.3, -0.3)
        adjusted_mean = mean + gender_adjustment
        
        bmis[mask] = np.clip(np.random.normal(adjusted_mean, std, size=mask.sum()), 15, 50)
    
    return np.round(bmis, 1)

# Generate age distribution based on defined proportions
def generate_age_distribution(num_rows):
    age_distribution = []
    for (lower, upper), proportion in age_groups.items():
        age_distribution.extend([age for age in range(lower, upper + 1)] * int(proportion * num_rows))
    return age_distribution

def generate_batch(num_rows, batch_id):
    age_distribution = generate_age_distribution(num_rows)
    ages = [np.random.choice(age_distribution) for _ in range(num_rows)]
    genders = np.random.choice(['Male', 'Female'], num_rows)
    
    df_batch = pd.DataFrame({'Age': ages, 'Gender': genders})
    df_batch['BMI'] = generate_bmi_vectorized(df_batch['Age'].values, df_batch['Gender'].values)
    
    # Save the batch to a CSV file
    file_name = f'synthetic_batch_{batch_id}.csv'
    df_batch.to_csv(file_name, index=False)
    print(f"Batch {batch_id} saved as {file_name}")

# Generate 50 batches of 5,000 rows each to reach 250,000 rows
start_time = time.time()

for i in range(50):  # 50 batches of 5,000 rows
    print(f"Generating batch {i+1} of 50...")
    generate_batch(5000, i + 1)

end_time = time.time()

# Total time taken
print(f"Time taken to generate 250,000 rows in 50 batches: {end_time - start_time:.2f} seconds")


#### **Combining Synthetic Dataset Batches**
The following code efficiently combines multiple batches of CSV files into one final dataset. This is necessary when working with large synthetic datasets that are generated in smaller chunks to avoid memory constraints. The process involves reading 50 individual CSV files, each containing a batch of 5,000 rows, and concatenating them into one comprehensive dataset of 250,000 rows. Once combined, the final dataset is saved as a new CSV file. This method ensures that large-scale data generation can be managed efficiently in smaller steps while maintaining the integrity of the final dataset. The code makes use of the pandas library to handle data manipulation and the os library to manage file paths and operations across the system.

In [102]:
import pandas as pd
import os

# Correct directory path where your CSV files are stored
directory = '/Users/steventuschman/Desktop/DATASET_250K'

# Initialize an empty list to store the DataFrames
df_list = []

# Loop through the files and read each CSV, appending them to the list
for i in range(1, 51):  # There are 50 files
    file_path = os.path.join(directory, f'synthetic_batch_{i}.csv')
    df_list.append(pd.read_csv(file_path))

# Concatenate all DataFrames into one
combined_df = pd.concat(df_list, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv(os.path.join(directory, 'combined_synthetic_dataset_250k.csv'), index=False)

print("All batches combined successfully into 'combined_synthetic_dataset_250k.csv'.")


All batches combined successfully into 'combined_synthetic_dataset_250k.csv'.


#### **Adding Waist Circumference to the Combined Dataset**

This code enhances the synthetic dataset by calculating a realistic waist circumference value based on BMI, age, and gender for each of the 250,000 individuals. The function takes into account gender-specific differences, age adjustments, and random variations to simulate natural variability in waist circumference. Once the waist circumference values are generated for all individuals, the updated dataset is saved as a new CSV file. This step is critical for analyzing risk factors associated with metabolic syndrome, as waist circumference is a key metric. The process ensures that the dataset remains consistent with real-world health data distributions.

In [103]:
import pandas as pd
import numpy as np

# Load the combined 250,000-row dataset
df_combined = pd.read_csv('combined_synthetic_dataset_250k.csv')

# Function to generate waist circumference
def generate_waist_circumference(bmi, age, gender):
    if gender == 'Male':
        base_wc = (bmi * 0.8) + 12  # Increased BMI impact for men
    else:
        base_wc = (bmi * 0.7) + 10  # Increased BMI impact for women
    
    # Age adjustment (slight increase with age)
    age_factor = (age - 30) / 100  # Increased age impact
    base_wc += base_wc * max(0, age_factor)
    
    # Add random variation (±3 inches)
    wc = base_wc + np.random.uniform(-3, 3)
    
    # Ensure waist circumference is within realistic bounds
    if gender == 'Male':
        wc = max(30, min(65, wc))  # Increased range for men
    else:
        wc = max(26, min(60, wc))  # Increased range for women
    
    return round(wc, 1)

# Apply the function to generate waist circumference for all 250,000 rows
df_combined['Waist_Circumference'] = df_combined.apply(lambda row: generate_waist_circumference(row['BMI'], row['Age'], row['Gender']), axis=1)

# Save the updated dataset with the new Waist Circumference column
df_combined.to_csv('combined_synthetic_dataset_250k_with_wc.csv', index=False)

print("Updated dataset saved successfully as 'combined_synthetic_dataset_250k_with_wc.csv'.")


Updated dataset saved successfully as 'combined_synthetic_dataset_250k_with_wc.csv'.


#### **Adding Fasting Blood Glucose (FBG) to the Combined Dataset**

In this code block, we further enrich the synthetic dataset by generating Fasting Blood Glucose (FBG) levels for each of the 250,000 individuals. The FBG values are calculated based on both age and BMI. A custom probability model determines the likelihood of elevated FBG levels (≥100 mg/dL) based on an individual’s BMI category (normal weight, overweight, or obese). The probability is then adjusted based on the individual's age, simulating the increased risk of high FBG as age progresses. This probabilistic method ensures the generated FBG values align with real-world health trends. Once calculated, the FBG values are added to the dataset, and the updated dataset is saved as a new CSV file for further analysis.

In [104]:
import pandas as pd
import numpy as np
import random

# Load the combined dataset
df_combined = pd.read_csv('combined_synthetic_dataset_250k_with_wc.csv')

# Function to categorize BMI
def get_bmi_category(bmi):
    if 18.5 <= bmi < 25:
        return "Normal weight"
    elif 25 <= bmi < 30:
        return "Overweight"
    elif bmi >= 30:
        return "Obese"
    else:
        return "Underweight"

# Function to get the base probability of having elevated FBG based on BMI category
def get_base_probability(bmi_category):
    probabilities = {
        "Normal weight": 0.15,
        "Overweight": 0.35,
        "Obese": 0.50
    }
    return probabilities.get(bmi_category, 0.15)

# Function to generate Fasting Blood Glucose (FBG) levels based on age and BMI
def generate_fbg(age, bmi):
    bmi_category = get_bmi_category(bmi)
    base_prob = get_base_probability(bmi_category)
    
    # Adjust probability with age factor
    age_factor = (age - 18) / (90 - 18)
    prob = base_prob + (0.1 * age_factor)  # Increase base probability with age factor
    
    if random.random() < prob:
        # Generate FBG for FBG >= 100 mg/dL
        return round(random.uniform(100, 300))
    else:
        # Generate FBG for normal range
        return round(random.uniform(70, 99))

# Apply the function to generate FBG for all rows in the dataset
df_combined['FBG'] = df_combined.apply(lambda row: generate_fbg(row['Age'], row['BMI']), axis=1)

# Save the updated dataset with the new FBG column
df_combined.to_csv('combined_synthetic_dataset_250k_with_fbg.csv', index=False)

print("Updated dataset saved successfully with FBG values as 'combined_synthetic_dataset_250k_with_fbg.csv'.")


Updated dataset saved successfully with FBG values as 'combined_synthetic_dataset_250k_with_fbg.csv'.


#### **Adding Triglyceride Levels to the Combined Dataset**

This code block introduces triglyceride levels for the individuals in the dataset, based on their BMI category and age. The probability of having elevated triglycerides (levels above 150 mg/dL) increases with both higher BMI categories and age, reflecting real-world trends where overweight and obese individuals are more likely to have elevated triglyceride levels. The triglyceride values are generated using random sampling within predefined ranges for each BMI category, adjusted by an age factor. After calculating these values, the updated dataset is saved as a new CSV file, ensuring the inclusion of triglyceride data for further analysis.

In [105]:
import numpy as np
import pandas as pd

# Define parameters for triglyceride levels based on BMI category
triglyceride_params = {
    'Underweight': {'range': (30, 200), 'prob_over_150': 0.05},
    'Normal weight': {'range': (40, 250), 'prob_over_150': 0.10},
    'Overweight': {'range': (50, 300), 'prob_over_150': 0.25},
    'Obese': {'range': (60, 350), 'prob_over_150': 0.35}
}

# Function to get BMI category
def get_bmi_category(bmi):
    if 18.5 <= bmi < 25:
        return "Normal weight"
    elif 25 <= bmi < 30:
        return "Overweight"
    elif bmi >= 30:
        return "Obese"
    else:
        return "Underweight"

# Function to generate triglyceride levels based on age and BMI category
def generate_triglyceride(age, bmi_category):
    params = triglyceride_params.get(bmi_category, triglyceride_params['Normal weight'])
    
    # Adjust probability with age factor
    age_factor = (age - 18) / (90 - 18)
    prob = params['prob_over_150'] + (0.1 * age_factor)  # Increase base probability with age factor
    
    if np.random.rand() < prob:
        return round(np.random.uniform(151, params['range'][1]))
    else:
        return round(np.random.uniform(params['range'][0], 150))

# Load the dataset
df_combined = pd.read_csv('combined_synthetic_dataset_250k_with_wc.csv')

# Add BMI Category column
df_combined['BMI_Category'] = df_combined['BMI'].apply(get_bmi_category)

# Apply the function to generate triglyceride levels
df_combined['Triglyceride'] = df_combined.apply(lambda row: generate_triglyceride(row['Age'], row['BMI_Category']), axis=1)

# Save the updated dataset with triglyceride values
df_combined.to_csv('combined_synthetic_dataset_250k_with_triglycerides.csv', index=False)

print("Triglyceride values added and saved successfully as 'combined_synthetic_dataset_250k_with_triglycerides.csv'.")


Triglyceride values added and saved successfully as 'combined_synthetic_dataset_250k_with_triglycerides.csv'.


#### **Adding HDL Levels to the Combined Dataset**

This code block generates HDL cholesterol levels for each individual in the dataset, based on their gender, BMI category, and age. The HDL levels are stratified according to BMI and gender, with a higher probability of low HDL levels for individuals who are overweight or obese. Additionally, the age factor further adjusts the likelihood of having HDL levels below the recommended thresholds (40 mg/dL for males and 50 mg/dL for females). The updated dataset is saved with these HDL values, contributing to a comprehensive set of metabolic health indicators for further analysis and model training.

In [106]:
import numpy as np
import pandas as pd

# Define HDL parameters based on gender and BMI category
hdl_params = {
    'Male': {
        'Underweight': {'range': (30, 80), 'prob_under_threshold': 0.08},
        'Normal weight': {'range': (30, 80), 'prob_under_threshold': 0.15},
        'Overweight': {'range': (25, 70), 'prob_under_threshold': 0.25},
        'Obese': {'range': (20, 60), 'prob_under_threshold': 0.40},
    },
    'Female': {
        'Underweight': {'range': (30, 80), 'prob_under_threshold': 0.03},
        'Normal weight': {'range': (30, 80), 'prob_under_threshold': 0.05},
        'Overweight': {'range': (25, 70), 'prob_under_threshold': 0.08},
        'Obese': {'range': (20, 60), 'prob_under_threshold': 0.15},
    }
}

# Function to generate HDL levels based on age, gender, and BMI category
def generate_hdl(age, row):
    gender = row['Gender']
    category = row['BMI_Category']
    
    hdl_range = hdl_params[gender][category]['range']
    prob_under_threshold = hdl_params[gender][category]['prob_under_threshold']
    
    # Set HDL threshold (Male < 40 mg/dL, Female < 50 mg/dL)
    threshold = 40 if gender == 'Male' else 50
    
    # Adjust probability with age factor
    age_factor = (age - 18) / (90 - 18)
    prob = prob_under_threshold + (0.1 * age_factor)  # Increase probability with age factor
    
    if np.random.rand() < prob:
        # Generate HDL for under-threshold values
        hdl = np.random.uniform(hdl_range[0], threshold)
    else:
        # Generate HDL for normal range
        hdl = np.random.uniform(threshold, hdl_range[1])
    
    return round(hdl)

# Load the dataset
df_combined = pd.read_csv('combined_synthetic_dataset_250k_with_triglycerides.csv')

# Apply the function to generate HDL values
df_combined['HDL'] = df_combined.apply(lambda row: generate_hdl(row['Age'], row), axis=1)

# Save the updated dataset with HDL values
df_combined.to_csv('combined_synthetic_dataset_250k_with_hdl.csv', index=False)

print("HDL values added and saved successfully as 'combined_synthetic_dataset_250k_with_hdl.csv'.")


HDL values added and saved successfully as 'combined_synthetic_dataset_250k_with_hdl.csv'.


#### **Adding High Blood Pressure Status to the Dataset**

This code block assigns high blood pressure status to individuals in the dataset based on their age and BMI category. The probability of having high blood pressure increases for individuals who are overweight or obese, with further adjustments based on age. For each individual, a binary status (1 = yes, 0 = no) is assigned, indicating whether they have high blood pressure. This probabilistic approach mirrors real-world trends where age and BMI are key risk factors for hypertension. After assigning the values, the updated dataset is saved for future analysis, now including high blood pressure status as an additional health metric.

In [107]:
import numpy as np
import pandas as pd

# Define high blood pressure probabilities based on BMI category
high_bp_prob = {
    'Underweight': 0.10,
    'Normal weight': 0.20,
    'Overweight': 0.35,
    'Obese': 0.50
}

# Function to assign high blood pressure status based on age and BMI category
def assign_high_bp(age, row):
    category = row['BMI_Category']
    base_prob = high_bp_prob[category]
    
    # Adjust probability with age factor
    age_factor = (age - 18) / (90 - 18)
    prob = base_prob + (0.1 * age_factor)  # Increase base probability with age factor
    
    # Assign high blood pressure (1 = yes, 0 = no) based on probability
    if np.random.rand() < prob:
        return 1
    else:
        return 0

# Load the dataset with HDL values
df_combined = pd.read_csv('combined_synthetic_dataset_250k_with_hdl.csv')

# Apply the function to assign high blood pressure status
df_combined['High_Blood_Pressure'] = df_combined.apply(lambda row: assign_high_bp(row['Age'], row), axis=1)

# Calculate the percentage of individuals with high blood pressure
high_bp_percentage = df_combined['High_Blood_Pressure'].mean() * 100
print(f"Percentage of individuals with high blood pressure: {high_bp_percentage:.2f}%")

# Save the updated dataset with high blood pressure status
df_combined.to_csv('combined_synthetic_dataset_250k_with_bp.csv', index=False)

print("High blood pressure values added and saved successfully as 'combined_synthetic_dataset_250k_with_bp.csv'.")


Percentage of individuals with high blood pressure: 42.08%
High blood pressure values added and saved successfully as 'combined_synthetic_dataset_250k_with_bp.csv'.


#### **Saving Final Synthetic Dataset**

After adding all key health metrics (BMI, waist circumference, fasting blood glucose, triglycerides, HDL levels, and high blood pressure status) to the synthetic dataset, we now save the final version of the dataset as `final_synthetic_dataset_250k.csv`. This dataset includes 250,000 rows with realistic distributions of these health-related metrics, making it useful for advanced data engineering projects. 


In [108]:
df_combined.to_csv('final_synthetic_dataset_250k.csv', index=False)


#### **Archiving Older Dataset Files**

This code block is responsible for organizing and archiving older versions of the dataset files. It first checks if an 'archive' folder exists in the directory and creates it if necessary. Then, it iterates through a list of older dataset files (e.g., combined_synthetic_dataset_250k_with_wc.csv, combined_synthetic_dataset_250k_with_fbg.csv) and moves them into the 'archive' folder. This helps maintain a clean working directory by keeping only the most relevant or final dataset files, while still preserving previous versions for reference or backup purposes. The use of the shutil.move function ensures that these older files are safely relocated without being deleted, preserving them for future retrieval if needed.

In [109]:
import shutil
import os

# Create an 'archive' folder if it doesn't exist
if not os.path.exists('archive'):
    os.makedirs('archive')

# List of older files to move to the archive
older_files = ['combined_synthetic_dataset_250k_with_wc.csv', 'combined_synthetic_dataset_250k_with_fbg.csv']

for file in older_files:
    if os.path.exists(file):
        shutil.move(file, 'archive/' + file)
        print(f"Moved {file} to archive folder.")


Moved combined_synthetic_dataset_250k_with_wc.csv to archive folder.
Moved combined_synthetic_dataset_250k_with_fbg.csv to archive folder.


#### **Saving Final Synthetic Dataset**

This code block saves the final version of the 250,000-row synthetic dataset to a CSV file. The dataset, which has been processed and includes all necessary metabolic variables (e.g., waist circumference, fasting blood glucose, HDL cholesterol, triglycerides, high blood pressure).  The index=False argument ensures that the row indices are not included in the output file, keeping the dataset clean and suitable for further analysis or machine learning applications. This step is crucial for preserving the dataset in its final form after all transformations and calculations have been applied.

#### **Deleting Unnecessary Files**

This code block is designed to remove redundant files that are no longer needed after generating the final version of the synthetic dataset. The script identifies and deletes the batch files (synthetic_batch_1.csv to synthetic_batch_50.csv) as well as older versions of the combined datasets (combined_synthetic_dataset_250k_with_wc.csv, combined_synthetic_dataset_250k_with_fbg.csv, combined_synthetic_dataset_250k.csv). Once these files are deleted, only the Jupyter Notebook and the final dataset remain in the working directory, streamlining the project and freeing up storage space. This ensures that the project directory remains clean and organized, containing only the essential files.

In [ ]:
import os

# List all batch files and older combined datasets to delete
files_to_delete = [f'synthetic_batch_{i}.csv' for i in range(1, 51)] + [
    'combined_synthetic_dataset_250k_with_wc.csv',
    'combined_synthetic_dataset_250k_with_fbg.csv',
    'combined_synthetic_dataset_250k.csv'
]

# Delete the files
for file in files_to_delete:
    if os.path.exists(file):
        os.remove(file)
        print(f"Deleted {file}.")

print("Unnecessary files deleted. Only Jupyter Notebook and final dataset remain.")


#### **Add-on Code Cell for FBG Issue Resolution: Probabilistic Data Imputation**

This code block is an essential step in resolving the missing Fasting Blood Glucose (FBG) values in the dataset. The technique used here can be classified as **probabilistic data imputation**, a common method in data preprocessing, especially when synthesizing or augmenting datasets for machine learning and statistical analysis.

In this case, the FBG values are generated using a probability-based method that mimics real-world health trends. Here’s an expanded explanation of the key functionalities:

- **BMI Categorization:** The dataset classifies individuals into BMI categories: **Normal weight**, **Overweight**, **Obese**, and **Underweight**. These categories play a significant role in determining the likelihood of elevated FBG, which is essential for assessing metabolic health.
  
- **Base Probability Calculation:** Based on the BMI category, a base probability is assigned for each individual regarding the likelihood of elevated FBG (FBG ≥ 100 mg/dL). For example, individuals in the **Obese** category have a higher base probability of elevated FBG than those in the **Normal weight** category. This probability is further adjusted by age to reflect the increasing likelihood of metabolic issues as age increases.

- **FBG Generation:** Using the base probability, the code employs random sampling to determine whether an individual’s FBG is elevated or within the normal range. The generated FBG values reflect a range of possible outcomes based on both the individual’s BMI and age. If an individual’s random draw falls below the adjusted probability, they receive an FBG value indicative of elevated blood sugar levels, else they receive a normal FBG value. 

This technique is particularly valuable when working with synthetic datasets where some variables are either missing or need to be simulated based on other known factors. By employing probabilistic data imputation, the dataset maintains its realism and statistical integrity, making it suitable for **machine learning models**, **statistical analysis**, and **research** in health-related fields.

Once the FBG values are generated and integrated into the dataset, the final dataset is saved with the newly added FBG column, ensuring the dataset is comprehensive and complete.


In [112]:
import pandas as pd
import numpy as np
import random

# Load the final dataset
df_combined = pd.read_csv('final_synthetic_dataset_250k.csv')

# Function to categorize BMI
def get_bmi_category(bmi):
    if 18.5 <= bmi < 25:
        return "Normal weight"
    elif 25 <= bmi < 30:
        return "Overweight"
    elif bmi >= 30:
        return "Obese"
    else:
        return "Underweight"

# Function to get the base probability of having elevated FBG based on BMI category
def get_base_probability(bmi_category):
    probabilities = {
        "Normal weight": 0.15,
        "Overweight": 0.35,
        "Obese": 0.50
    }
    return probabilities.get(bmi_category, 0.15)

# Function to generate Fasting Blood Glucose (FBG) levels based on age and BMI
def generate_fbg(age, bmi):
    bmi_category = get_bmi_category(bmi)
    base_prob = get_base_probability(bmi_category)
    
    # Adjust probability with age factor
    age_factor = (age - 18) / (90 - 18)
    prob = base_prob + (0.1 * age_factor)  # Increase base probability with age factor
    
    if random.random() < prob:
        # Generate FBG for FBG >= 100 mg/dL
        return round(random.uniform(100, 300))
    else:
        # Generate FBG for normal range
        return round(random.uniform(70, 99))

# Apply the function to generate FBG for all rows in the dataset
df_combined['FBG'] = df_combined.apply(lambda row: generate_fbg(row['Age'], row['BMI']), axis=1)

# Save the updated dataset with the new FBG column
df_combined.to_csv('final_synthetic_dataset_250k.csv', index=False)

print("Updated dataset saved successfully with FBG values.")


Updated dataset saved successfully with FBG values.


#### **Conclusion**

This project successfully generated a comprehensive synthetic dataset of 250,000 adults, which mirrors the real-world health metrics of the U.S. adult population. By leveraging the detailed data distributions from NHANES (CDC), we created a dataset that accurately reflects key metabolic health variables such as Age, BMI, Waist Circumference, Fasting Blood Glucose (FBG), Triglycerides, HDL Cholesterol, and High Blood Pressure.

Each health metric was crafted using a combination of probabilistic models, age and gender adjustments, and algorithmic simulations. This process ensured that the data closely aligns with real-world distributions, creating a valuable resource for data engineering, machine learning, and health research projects. The large size of the dataset and its realistic variability offer significant potential for training machine learning models and conducting simulations in a privacy-compliant way.

Through the use of advanced algorithms, this dataset not only provides a foundation for research into metabolic syndrome and associated health risks but also addresses the gap in the availability of health-related synthetic data that maintains real-world fidelity. By making such data readily accessible, this project can support a wide range of applications in healthcare, public health, insurance, and academic research, offering a realistic and scalable alternative to sensitive real-world health datasets.

The final product, a clean, 250,000-row synthetic dataset, stands as a key asset for organizations looking to explore, predict, and model metabolic health conditions under realistic conditions, free from privacy concerns.